# Instalando dependências

In [1]:
!pip install parsel requests pandas openpyxl


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Importando dependências

In [3]:
import requests
import parsel
import os
import re
import time
import pandas as pd

# Funções

# Funções auxiliares

In [32]:
def trata_cpf(string):
    try:
        padrao_cep = r'\b\d{5}-\d{3}\b'
        cep_encontrado = re.findall(padrao_cep, string)[0]
    except Exception as E:
        padrao_cep = r'\b\d{8}\b'
        cep_encontrado = re.findall(padrao_cep, string)[0]

    return cep_encontrado


def consultar_cep(cep):
    cep = ''.join(filter(str.isdigit, str(cep)))
    url = f'https://brasilapi.com.br/api/cep/v1/{cep}'
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f'Erro ao consultar o CEP {cep}: Status Code {response.status_code}')
            return None
    except Exception as e:
        print(f'Erro ao consultar o CEP {cep}: {e}')
        return None
    
def gera_planilha(dados, nome_planilha):
    df = pd.DataFrame(dados)
    df.to_excel(f'{nome_planilha}.xlsx', index=True)

## Função que pega a ultima página e gera os links das páginas "Principais" 

In [5]:
def ultima_pagina():
    resposta = requests.get('https://sga.softexrecife.org.br/catalogo')
    html_selector = parsel.Selector(resposta.text)
    last_range = html_selector.xpath("//ul[@class='pagination pagination']/li[13]/a/text()").get()

    return last_range


def gera_links():    

    links_paginas = []

    for pagina in range(1,int(ultima_pagina())+1):
        url = f'https://sga.softexrecife.org.br/catalogo?page={pagina}'
        links_paginas.append(url)
    
    return links_paginas


## Função que agrupa os links de cada card

In [6]:
def agrupa_links(links):
    url = 'https://sga.softexrecife.org.br/catalogo'
    cards_url = []
    for link in links:
        cards_url.append(url + '/' + os.path.basename(link))
    return  cards_url

## Função que pega os dados da startup na página referente

In [37]:
def dados_startup(link):
    try:
        resposta = requests.get(link)
        cards_html = resposta.text
        cards_selector = parsel.Selector(cards_html)

        telefone = cards_selector.xpath("//div[@class='col-contact']//div[@class='col-content']/a[1]/text()").get()
        email = cards_selector.xpath("//div[@class='col-contact']//div[@class='col-content']/a[2]/text()").get()
        site = cards_selector.xpath("//div[@class='col-contact']//div[@class='col-content']/a[3]/text()").get()
        endereco = cards_selector.xpath("//div[@class='col-contact']//div[@class='col-content']/a[4]/text()").get().split()
        endereco = ' '.join(endereco)
        try:
            cep = trata_cpf(endereco)
            cidade = consultar_cep(cep).get('city')
        except Exception as Er:
            print(Er,'- Error')
        nome_startup = cards_selector.xpath("//div[@class='col-details']/p/text()").get().strip()
        descricao_startup = cards_selector.xpath("//div[@class='col-details']/textarea/text()").get().split()
        descricao_startup = ' '.join(descricao_startup)
        segmento_startup = cards_selector.xpath("//div[@class='col-details']/p[3]/text()").get()
        modelo_startup = cards_selector.xpath("//div[@class='col-details']/p[5]/text()").get()

        dados = {
            'Nome': nome_startup,
            'Segmento': segmento_startup,
            'Modelo': modelo_startup,
            'Site': site,
            'Email': email,
            'Telefone': telefone,
            'Endereço': endereco,
            'Cidade': cidade,
            'Descrição': descricao_startup,
        }

        return dados

                
    except Exception as E:
        print(f'Erro ao acessar o site: {E}, {link}')

## Função que percorre todas as páginas, e chama a função que percorre as informações nos cards.

In [8]:
def percorre_startups(links_paginas):
    dados = []
    caminho_xpath = "//div[@class='catalog-content-main-company border cl-page-width rounded row d-flex align-items-center justify-content-center']/a/@href"
    for pagina in links_paginas:
        time.sleep(1)
        print(f'Pagina - {pagina}')
        resposta = requests.get(pagina)
        html_text = resposta.text
        html_selector = parsel.Selector(html_text)
        links = html_selector.xpath(caminho_xpath).getall()
        links = agrupa_links(links)
        for link in links:
            resposta = dados_startup(link)
            if resposta == None:
                continue
            else:
                dados.append(resposta)
        
    return dados


## Executando

In [10]:
links_principais = gera_links()

In [39]:
dados = percorre_startups(links_principais)

Pagina - https://sga.softexrecife.org.br/catalogo?page=1
Erro ao consultar o CEP 50030600: Status Code 404
'NoneType' object has no attribute 'get' - Error
Erro ao acessar o site: cannot access local variable 'cidade' where it is not associated with a value, https://sga.softexrecife.org.br/catalogo/449
Pagina - https://sga.softexrecife.org.br/catalogo?page=2
Pagina - https://sga.softexrecife.org.br/catalogo?page=3
Pagina - https://sga.softexrecife.org.br/catalogo?page=4
Pagina - https://sga.softexrecife.org.br/catalogo?page=5
Pagina - https://sga.softexrecife.org.br/catalogo?page=6
Pagina - https://sga.softexrecife.org.br/catalogo?page=7
Pagina - https://sga.softexrecife.org.br/catalogo?page=8
Pagina - https://sga.softexrecife.org.br/catalogo?page=9
list index out of range - Error
Erro ao acessar o site: cannot access local variable 'cidade' where it is not associated with a value, https://sga.softexrecife.org.br/catalogo/241
Pagina - https://sga.softexrecife.org.br/catalogo?page=10
li

In [40]:
gera_planilha(dados, 'Startup_Cidade')